# Assignment 2 Week 2

In [2]:
import sys, os
from datetime import datetime, timedelta,date
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import json
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model

### Load Data

In [3]:
products_df = pd.read_csv('./../data/amazon_baby_subset.csv')

In [4]:
with open('./../data/important_words.json', 'r') as f: # Reads the list of most frequent words
    important_words = json.load(f)
important_words = [str(s) for s in important_words]

In [5]:
print('# of positive reviews =', len(products_df[products_df['sentiment']==1]))
print('# of negative reviews =', len(products_df[products_df['sentiment']==-1]))

# of positive reviews = 26579
# of negative reviews = 26493


In [6]:
products_df.review = products_df.review.fillna('')
products_df['reviews_clean'] = products_df.review.str.replace('[^\w\s]','').fillna('')
for word in important_words:
    products_df[word] = products_df['reviews_clean'].apply(lambda s : s.split().count(word))


In [7]:
train_indexes = pd.read_json('../data/module-4-assignment-train-idx.json')
val_indexes = pd.read_json('../data/module-4-assignment-validation-idx.json')

In [9]:
train_data = products_df.iloc[train_indexes[0].tolist()]
val_data = products_df.iloc[val_indexes[0].tolist()]

In [10]:
def get_numpy_data(data_frame, features, label):
    """
    features: list of feature columns
    label: str label columns name
    """
    data_frame['intercept'] = 1
    features = ['intercept'] + features
    features_frame = data_frame[features]
    feature_matrix = features_frame.as_matrix()
    label_frame  = data_frame[label]
    label_array = label_frame.as_matrix()
    return(feature_matrix, label_array)

In [13]:
feature_matrix_train, sentiment_train = get_numpy_data(train_data, important_words, 'sentiment')
feature_matrix_valid, sentiment_valid = get_numpy_data(val_data, important_words, 'sentiment')

/Users/nehiljain/.pyenv/versions/3.4.6/envs/py34/lib/python3.4/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    scores = feature_matrix.dot(coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1.0/(1.0 + np.exp(-scores))
    
    # return predictions
    return predictions

In [15]:
def feature_derivative_with_L2(errors, feature, coefficient, l2_penalty, feature_is_constant): 
    derivative = np.dot(errors, feature)
    if not feature_is_constant:
        derivative = np.dot(errors, feature) - 2 * l2_penalty * coefficient
    return derivative
    

In [16]:
def compute_log_likelihood_with_L2(feature_matrix, sentiment, coefficients, l2_penalty):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores))) - l2_penalty*np.sum(coefficients[1:]**2)
    
    return lp

In [20]:
def logistic_regression_with_L2(feature_matrix, sentiment, initial_coefficients, step_size, l2_penalty, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_i,w) using your predict_probability() function
        ## YOUR CODE HERE
        predictions = predict_probability(feature_matrix, coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        for j in 
# run with L2 = 10
coefficients_10_penalty = logistic_regression_with_L2(feature_matrix_train, sentiment_train,
                                                      initial_coefficients=np.zeros(194),
                                                      step_size=5e-6, l2_penalty=10, max_iter=501)range(len(coefficients)): # loop over each coefficient
            is_intercept = (j == 0)
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j].
            # Compute the derivative for coefficients[j]. Save it in a variable called derivative
            ## YOUR CODE HERE
            derivative = feature_derivative_with_L2(errors, feature_matrix[:, j], coefficients[j], l2_penalty, is_intercept)
            
            
            # add the step size times the derivative to the current coefficient
            ## YOUR CODE HERE
            coefficients = coefficients + step_size * derivative
        
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood_with_L2(feature_matrix, sentiment, coefficients, l2_penalty)
            print('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

In [21]:
coefficients_0_penalty = logistic_regression_with_L2(feature_matrix_train, sentiment_train,
                                                     initial_coefficients=np.zeros(194),
                                                     step_size=5e-6, l2_penalty=0, max_iter=501)

iteration   0: log likelihood of observed labels = -38432.40613172
iteration   1: log likelihood of observed labels = -347079.20583932
iteration   2: log likelihood of observed labels = -270977.08597932
iteration   3: log likelihood of observed labels = -292539.75364679
iteration   4: log likelihood of observed labels = -320156.20846519
iteration   5: log likelihood of observed labels = -239770.07077246
iteration   6: log likelihood of observed labels = -367049.81808239
iteration   7: log likelihood of observed labels = -189344.44520061
iteration   8: log likelihood of observed labels = -410170.31294279
iteration   9: log likelihood of observed labels = -143256.74334393
iteration  10: log likelihood of observed labels = -445576.99758584
iteration  11: log likelihood of observed labels = -106062.47829825
iteration  12: log likelihood of observed labels = -466035.54319332
iteration  13: log likelihood of observed labels = -85177.65127790
iteration  14: log likelihood of observed labels =

In [ ]:
coefficients_10_penalty = logistic_regression_with_L2(feature_matrix_train, sentiment_train,
                                                      initial_coefficients=np.zeros(194),
                                                      step_size=5e-6, l2_penalty=10, max_iter=501)
coefficients_1e2_penalty = logistic_regression_with_L2(feature_matrix_train, sentiment_train,
                                                       initial_coefficients=np.zeros(194),
                                                       step_size=5e-6, l2_penalty=1e2, max_iter=501)
coefficients_1e3_penalty = logistic_regression_with_L2(feature_matrix_train, sentiment_train,
                                                       initial_coefficients=np.zeros(194),
                                                       step_size=5e-6, l2_penalty=1e3, max_iter=501)


iteration   0: log likelihood of observed labels = -38425.25480580
iteration   1: log likelihood of observed labels = -344048.39077454
iteration   2: log likelihood of observed labels = -275279.32666684
iteration   3: log likelihood of observed labels = -288357.94233088
iteration   4: log likelihood of observed labels = -324547.86231617
iteration   5: log likelihood of observed labels = -236637.14234075
iteration   6: log likelihood of observed labels = -369727.58802478
iteration   7: log likelihood of observed labels = -189270.02727123
iteration   8: log likelihood of observed labels = -409634.66047233
iteration   9: log likelihood of observed labels = -147772.38847181
iteration  10: log likelihood of observed labels = -441446.06015742
iteration  11: log likelihood of observed labels = -115211.99034563
iteration  12: log likelihood of observed labels = -460995.01246537
iteration  13: log likelihood of observed labels = -95637.72003658
iteration  14: log likelihood of observed labels =